# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using surprise. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives
You will be able to:
* Fit a recommender system model to a set of data
* Create a function that will return the top recommendations for a user
* Introduce a new user to a rating matrix and make recommendations for them

For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last  lesson, you got exposed to working with Surprise datasets. In this lab, you will also go through the process of reading in a dataset into the Surprise dataset format. To begin with, load the dataset into a pandas dataframe. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [1]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
userId       100836 non-null int64
movieId      100836 non-null int64
rating       100836 non-null float64
timestamp    100836 non-null int64
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [2]:
#drop unnecessary columns
new_df=df.drop('timestamp', axis=1)
new_df

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


It's now time to transform the dataset into something compatible with Surprise. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [3]:
!pip install surprise
from surprise import Reader, Dataset
# read in values as Surprise dataset 
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(new_df[['userId', 'movieId', 'rating']], reader)

     |████████████████████████████████| 6.5MB 22.2MB/s eta 0:00:01
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1709103 sha256=050d4c985a73db1b070dfbe068547f00a87b3f3cec98b6d0b370e590117ceb29
  Stored in directory: /home/secure-counter-6252/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [4]:
dataset = data.build_full_trainset()
print('Number of users: ',dataset.n_users,'\n')
print('Number of items: ',dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the Best Model
Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [5]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [6]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run
params = {'n_factors' :[20,50,100],
         'reg_all':[0.02,0.05,0.1]}
grid_search = GridSearchCV(SVD,param_grid=params,n_jobs=1)
grid_search.fit(data)

In [8]:
# print out optimal parameters for SVD after GridSearch
print(grid_search.best_params)
print(grid_search.best_score)

{'rmse': {'n_factors': 20, 'reg_all': 0.05}, 'mae': {'n_factors': 20, 'reg_all': 0.05}}
{'rmse': 0.8694778013575688, 'mae': 0.6687148457704131}


In [9]:
# cross validating with KNNBasic
knn_basic = KNNBasic(sim_options={'name':'pearson','user_based':True})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)

In [12]:
# print out the average RMSE score for the test set
cv_knn_basic['test_rmse'].mean()

0.9744296727280544

In [13]:
# cross validating with KNNBaseline
knn_baseline = KNNBaseline(sim_options={'name':'pearson','user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data,n_jobs=-1)

In [14]:
# print out the average score for the test set
cv_knn_baseline['test_rmse'].mean()

0.8764505389253546

Based off these outputs, it seems like the best performing model is the SVD model with n_factors = 50 and a regularization rate of 0.05. Let's use that model to make some predictions. Use that model or if you found one that performs better, feel free to use that.

## Making Recommendations

This next section is going to involve making recommendations, and it's important that the output for the recommendation is interpretable to people. Rather than returning the movie_id values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [8]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [9]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [10]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [11]:
svd.predict(2,4)

Prediction(uid=2, iid=4, r_ui=None, est=3.017488620249605, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put all of our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step to go let's create a function that allows students to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater` should take as parameters:
* movie_df : DataFrame - a dataframe containing the movie ids, name of movie, and genres
* num : int - number of ratings
* genre : string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int  , 'movieId': int  ,'rating': float  }

#### This function is optional, but fun :) 

In [36]:
def movie_rater(movie_df,num, genre=None):
    
        

In [37]:
# try out the new function here!

      movieId                   title          genres
6579    55245  Good Luck Chuck (2007)  Comedy|Romance
How do you rate this movie on a scale of 1-5, press n if you have not seen :
5
      movieId                       title          genres
1873     2491  Simply Irresistible (1999)  Comedy|Romance
How do you rate this movie on a scale of 1-5, press n if you have not seen :
4
      movieId                  title  genres
3459     4718  American Pie 2 (2001)  Comedy
How do you rate this movie on a scale of 1-5, press n if you have not seen :
4
      movieId             title                   genres
4160     5990  Pinocchio (2002)  Children|Comedy|Fantasy
How do you rate this movie on a scale of 1-5, press n if you have not seen :
3


If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [12]:
user_rating = [{'userId': 1000, 'movieId': 55245, 'rating': '5'},
 {'userId': 1000, 'movieId': 2491, 'rating': '4'},
 {'userId': 1000, 'movieId': 4718, 'rating': '4'},
 {'userId': 1000, 'movieId': 5990, 'rating': '3'}]

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a Surprise dataset
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number)

In [13]:
## add the new ratings to the original ratings DataFrame
new_ratings_df = new_df.append(user_rating,ignore_index=True)
new_data = Dataset.load_from_df(new_ratings_df,reader)

In [14]:
# train a model using the new combined DataFrame
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(new_data.build_full_trainset())

In [16]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
list_of_movies = []
for m_id in new_df['movieId'].unique():
    list_of_movies.append( (m_id,svd.predict(1000,m_id)[3]))

list_of_movies

[(1, 4.1157493754250485),
 (3, 3.544969504619686),
 (6, 4.1296955223030825),
 (47, 4.185919578458682),
 (50, 4.502860705319763),
 (70, 3.722141978701826),
 (101, 4.085795460896009),
 (110, 4.281008858123594),
 (151, 3.753845897997472),
 (157, 3.3998942017828666),
 (163, 3.7974058101348835),
 (216, 3.6215417525048172),
 (223, 4.087953065539062),
 (231, 3.25461337068125),
 (235, 3.8427476675988),
 (260, 4.403648899425609),
 (296, 4.415979256356789),
 (316, 3.491274949282414),
 (333, 3.9421691324305725),
 (349, 3.781965392732843),
 (356, 4.337162926957547),
 (362, 3.759458958761461),
 (367, 3.3578560777539677),
 (423, 3.4036782176707954),
 (441, 4.206709261265389),
 (457, 4.197567929529437),
 (480, 3.951070195844506),
 (500, 3.646159889595878),
 (527, 4.573463030989544),
 (543, 3.596497322712711),
 (552, 3.4288981371995377),
 (553, 3.9403236536458706),
 (590, 3.9996091716249733),
 (592, 3.543752393908982),
 (593, 4.425644482331933),
 (596, 3.728864362967354),
 (608, 4.30935024916122),
 (6

In [17]:
# order the predictions from highest to lowest rated
sorted(list_of_movies, key=lambda x: x[1], reverse = True)
ranked_movies = sorted(list_of_movies, key=lambda x: x[1], reverse = True)
ranked_movies

[(318, 4.704512546522993),
 (750, 4.592043154568701),
 (527, 4.573463030989544),
 (1204, 4.532294379775931),
 (2959, 4.532270086489676),
 (1248, 4.504794891512441),
 (50, 4.502860705319763),
 (720, 4.494358393896519),
 (858, 4.492547433035978),
 (475, 4.490341698676106),
 (912, 4.462931209995107),
 (1136, 4.4593923384040215),
 (904, 4.447757830473101),
 (741, 4.443233620619136),
 (58559, 4.44287493610382),
 (4973, 4.440389758751774),
 (2019, 4.439466951398066),
 (1213, 4.432245056520899),
 (56782, 4.431508143058214),
 (1276, 4.43020805603695),
 (1221, 4.428170071855077),
 (1104, 4.426201345446598),
 (593, 4.425644482331933),
 (898, 4.420025713952553),
 (1201, 4.418982096005559),
 (3451, 4.417333608568928),
 (3435, 4.416638148830045),
 (296, 4.415979256356789),
 (2324, 4.415904629047624),
 (38061, 4.413644363752382),
 (1208, 4.41004676143935),
 (2329, 4.406084050529361),
 (48516, 4.405405277841126),
 (1148, 4.40367878934087),
 (260, 4.403648899425609),
 (1089, 4.402897350467427),
 (1262

 For the final component of this challenge, it could be useful to create a function `recommended_movies` that takes in the parameters:
* `user_ratings` : list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df` : DataFrame 
* `n` : int- number of recommended movies 

The function should use a for loop to print out each recommended *n* movies in order from best to worst

In [36]:
# return the top n recommendations using the 
def recommended_movies(user_ratings,movie_title_df,n):
        for idx, rec in enumerate(user_ratings):
            title = movie_title_df.loc[movie_title_df['movieId'] == int(rec[0])]['title']
            print('Recommendation # ',idx+1,': ',title,'\n')
            n-= 1
            if n == 0:
                break
            
recommended_movies(ranked_movies,df_movies,5)

Recommendation #  1 :  277    Shawshank Redemption, The (1994)
Name: title, dtype: object 

Recommendation #  2 :  602    Dr. Strangelove or: How I Learned to Stop Worr...
Name: title, dtype: object 

Recommendation #  3 :  841    Streetcar Named Desire, A (1951)
Name: title, dtype: object 

Recommendation #  4 :  906    Lawrence of Arabia (1962)
Name: title, dtype: object 

Recommendation #  5 :  3622    Amelie (Fabuleux destin d'Amélie Poulain, Le) ...
Name: title, dtype: object 



## Level Up

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top n recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the change to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will get exposed to using spark to make recommender systems.